In [1]:
# Requirements
# Python 3
# Python libraries: pandas

# Can be downloaded from python.org
# pandas can be downloaded by running 'pip install pandas' from the terminal 
# (may need to use pip3 instead of pip)


# How To Use

# Download zip and unzip it. 
# Run setup script: hit shift and right-click in the unzipped folder and click on 'Open PowerShell window here'.
# Type `python fgstSetup.py` in the terminal and wait for it to complete. 
# Run data collection script in PowerShell after each session by running `python fallGuysData.py` 
# (May need to use python3 instead of python)

# Note: 
# If you forget to run for a session, run 'python fallGuysData.py Player-prev.log' 
# before running python fallGuysData.py for your new session
# word that better

In [2]:
# Things to do...

# get script to run when Player.log is updated...or just run after each session
# fix variable naming convention to use _s
# new repo for it

In [3]:
import os, datetime, time
import pandas as pd

# https://stackoverflow.com/a/10854983
offset = time.timezone if (time.localtime().tm_isdst == 0) else time.altzone
tz = offset / 60 / 60 
HOURS_DIFFERENTIAL = int(tz) # set time zone each time

# need hex-a-gone and royal fumble
list_of_finals = ['round_fall_mountain', 'round_floor_fall', 'round_royalfumble', 'round_jump_showdown', 
                  'round_fall_mountain_hub_complete']

with open('totalshows.txt') as f:
    total_shows = f.read()
total_shows = int(total_shows.strip())

with open(os.path.join('data', 'session.txt')) as f:
    session_num = f.read()
session_num = int(session_num)

In [4]:
# Notes

# find player id for round - don't think we can do anything about that
# use that as well instead of username
# only affects userEndRoundLines

# check elim time for jump_showdown - seems as though personal end times are only for...
# ...finishing races early

# playing around Daylight Savings switch may result in bad times for that round

# figure out if there's a max number of shows that will be stored in the .log : seems the whole session

In [5]:
def cleanLines(lines):
    return [line.replace('[', '').replace(']', '').replace('>', '').strip() for line in lines]

# splits show into show data and rounds
def roundSplit(lines):
    lines = cleanLines(lines)
    splits = []
    # start of each round in highlights section
    for i, line in enumerate(lines):
        if 'Round' in line:
            splits.append(i)
    
    splits.append(0)
    # split it into rounds
    rounds = [lines[splits[i-1]:splits[i]] if i != len(splits)-1 else lines[splits[i-1]:]
              for i in range(1, len(splits))]
    
    return lines[1:splits[0]], rounds

# takes registered and connected lines
# returns start times for rounds
# (time at which game is found)
def getStartTimes(reg, conne):
    startTimes = []
    
    for i in range(len(conne)):
        registeredTime = datetime.datetime.strptime(reg[i].split('at: ')[-1], '%m/%d/%Y %I:%M:%S %p')
        connectedTime = datetime.datetime.strptime(conne[i].split(': [')[0], '%H:%M:%S.%f')
        
        d = (connectedTime - registeredTime)
        startTimes.append(registeredTime + (d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL)))
    
    return startTimes

def getTimeTaken(start, end):
    d = datetime.datetime.strptime(end, '%H:%M:%S.%f') - start
    # start time was already adjusted for HOURS_DIFFERENTIAL, so need to again
    return str(d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL))[2:] # so hours isn't included

# edit this later change
# grab the last Seasonthing in txt file - countdown in .log
# if its larger than the last one, then the season reset
def getSeason():
    return 2

# gets lines for CompletedEpisode section of a show
def getShowLines(lines, marker):
    finalLines = []
    tempLines = lines[marker:]
    for line in tempLines:
        if line == '':
            continue
        if '>' not in line and '[Round' not in line and '[Complet' not in line:
            break
        if line != "":
            finalLines.append(line)
    return finalLines

def getExtraRoundInfoLines(possLines): # rename?
    rnds = []
    currRnd = possLines[0][1].split()[0]
    currSID = possLines[0][0]
    prevLine = possLines[0][2]
    
    # get last line before map switches (and server ID)
    for i, (serverID, line, line_num) in enumerate(possLines):
        rnd = line.split()[0]
        
        if rnd != currRnd: 
            currRnd = rnd
            currSID = serverID
            rnds.append(possLines[i-1][1])
            if (prevLine + 1) == line_num: # sometimes server changes map due to a dropout
                rnds.pop()      
        elif serverID != currSID:
            currRnd = rnd
            currSID = serverID
            rnds.append(possLines[i-1][1])
            if (prevLine + 1) == line_num:
                rnds.pop()
        prevLine = line_num
    
    rnds.append(possLines[-1][1])      
    return rnds


# preprocessGrade1 has been retired
# remove lines in-between completed games
def preprocessGrade2(lines):
    print(len(lines))
    lines2 = []
    allGood = False
    badgeID = False
    for line in lines:
        if '[CATAPULT] Login Succeeded' in line:
            allGood = True
        if 'BadgeId:' in line:
            badgeID = True
        if badgeID:
            if '[ClientGlobalGameState] ShutdownNetworkManager' in line or '[ClientGlobalGameState] sending graceful disconnect message' in line:
                badgeID = False
                allGood = False
                lines2.append(line)

        if allGood:
            lines2.append(line)
    lines = lines2
    print(len(lines))
    return lines

# changes to the data extraction removed the need for this
def preprocessGrade3(lines):
    #print(len(lines))
    start_conn = -1
    to_remove = []
    in_conn = False
    for i, line in enumerate(lines):
        if "[StateConnectToGame] We're connected to the server!" in line:
            start_conn = i
            in_conn = True
        if 'reports that it is not yet ready to accept connections.' in line:
            if in_conn:
                to_remove.append([start_conn, i])
                in_conn = False
    
    temp_lines = []
    for i, line in enumerate(lines):
        for check in to_remove:
            if i >= check[0] and i <= check[1]:
                continue
            temp_lines.append(line)
            
    #print(len(temp_lines))
    return temp_lines
    
# remove spectated rounds
def preprocessGrade4(lines):
    #print(len(lines))
    start_round = -1
    to_remove = []
    in_spec = False
    for i, line in enumerate(lines):
        if "Received instruction that server is ending a round, and to rejoin" in line:
            start_round = i
        if 'permission=Spectator' in line:
            in_spec = True
        if '[ClientGameManager] Shutdown' in line:
            if in_spec:
                to_remove.append([start_round, i])
                in_spec = False
                
    # remove selected lines
    temp_lines = []
    for i, line in enumerate(lines):
        to_append = True
        for check in to_remove:
            if i >= check[0] and i <= check[1]:
                to_append = False
        if to_append:
            temp_lines.append(line)
    
    #print(len(temp_lines))
    return temp_lines

# gets rid of shows in which the user got disconnected ()
def preprocessGrade5(lines):
    #print(len(lines))
    
    start_conn = -1
    to_remove = []
    in_conn = False # really just after first show starts
    end_ep = False
    
    for i, line in enumerate(lines):
        if '[CATAPULT] Login Succeeded' in line:
            if in_conn and not end_ep:
                to_remove.append([start_conn, i-1])
            start_conn = i
            in_conn = True
            end_ep = False
            
        if '[CompletedEpisodeDto]' in line:
            end_ep = True
                
    # remove selected lines
    temp_lines = []
    for i, line in enumerate(lines):
        to_append = True
        for check in to_remove:
            if i >= check[0] and i <= check[1]:
                to_append = False
        if to_append:
            temp_lines.append(line)
         
    #print(len(temp_lines))
    return temp_lines

# helper function (to account for if round wraps around midnight)
def subtractHours(a, b):
    c = (datetime.datetime.strptime(b, '%H:%M:%S.%f') - datetime.datetime.strptime(a, '%H:%M:%S.%f'))
    return str(c - datetime.timedelta(days=c.days))

# save data in csvs
def saveData(show, roundsList):
    new_shows_df = pd.DataFrame(pd.Series(show)).T
    new_rounds_df = pd.DataFrame(roundsList)
    try:
        #load them
        shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
        rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
        
        if str(show['Start Time'])[:21] in [tm[:21] for tm in shows_df['Start Time'].tolist()]:
            return False
        
        # append
        shows_df = shows_df.append(new_shows_df, ignore_index=True)
        rounds_df = rounds_df.append(new_rounds_df, ignore_index=True)
    except: # first time
        shows_df = new_shows_df
        rounds_df = new_rounds_df
        print('first time...creating csvs')
    
    # write them to their respective files
    shows_df.to_csv(os.path.join('data', 'shows.csv'), index=False)
    rounds_df.to_csv(os.path.join('data', 'rounds.csv'), index=False)
    return True

In [6]:
with open("log_path.txt") as f:
    log_path = f.read()

with open(log_path) as f:
    lines = f.read()

#with open(os.path.join('C:\\Users','Joseph','Downloads','Player (1).log')) as f:
 #   lines = f.read()
    
#with open('personal\\data\\archive\\session11.txt') as f:
 #   lines = f.read()

lines = lines.split('\n')
#lines = preprocessGrade2(lines)
print(len(lines))
lines = preprocessGrade4(lines)
print(len(lines))
lines = preprocessGrade5(lines)
print(len(lines))

67642
67252
64244


In [7]:
# get first line of each new show and usernames used for show
prevUser = '!!!!!!!!!!!!!!!'
lookUser = True
inRound = False
inARound = False
num_players_lock = False
gameMode = 'main_show'

episodeMarkers = []
usernames = []
partySizes = []

# times
reg = []
conne = []
startRoundLines = []
userEndRoundLines = []
actualEndRoundLines = []

possLines = []
gameModes = []
# to find the actual number of players that qualified (for racing rounds)
prevNumLine = "green"
numLines = []

# **********************************************************
# go through lines, looking for certain things**************
# **********************************************************
for i, line in enumerate(lines):
    # for username
    if lookUser and 'Sending login request' in line:
        usernames.append(line.split(' player ')[-1].split(' networkID')[0].replace(',', ''))
        prevUser = usernames[-1]
        lookUser = False
    # for type of show (main or alternate) (also called playlist)
    elif 'Chosen Show:' in line: # appears before entering matchmaking solo/group
        gameMode = line.split(':')[-1]
    # signifies start of looking for new episode
    # get size of party
    elif 'Party Size' in line or 'Begin matchmaking solo' in line:
        if 'Begin matchmaking solo' in line:
            partySize = 1
        else:
            partySize = int(line.split(' ')[-1].strip())
    # for show start time
    elif '[QosManager] Registered' in line or 'QosManager: Registered' in line: # for registered time (date)
        to_add_reg = line
    elif "[StateConnectToGame] We're connected to the server!" in line: # for connection time
        to_add_conne = line
    # for server ID and map lines
    elif 'Received NetworkGameOptions from ' in line: 
        tmp = line.split('roundID=')[-1]
        serverID = line.split(' ')[4]
        if 'Default' not in tmp:
            possLines.append([serverID, tmp, i])
    # for start round times and players that qualified from previous round
    elif 'state from Countdown to Playing' in line:
        startRoundLines.append(line.split(': [')[0])
        inRound = True
        inARound = True
        num_players_lock = False
        # append last # players achieving obj when hit new round
        if prevNumLine != "green":
            numLines.append(prevNumLine.split('=')[-1])
            prevNumLine = ""
    elif 'Changing state from GameOver to Results' in line: # occassionally a random NumPlayers... line right before new round
        num_players_lock = True
    elif '[ClientGameSession] NumPlayersAchievingObjective=' in line: # for total number of players that quality
        if not num_players_lock:
            prevNumLine = line
    # for end round / player active in round times
    elif '[ClientGameManager] Handling unspawn for player FallGuy' in line and prevUser in line:
        if inRound:
            userEndRoundLines.append(line.split(': [')[0])
            inRound = False
    elif 'Changing local player state to: SpectatingEliminated' in line:
        if inRound:
            userEndRoundLines.append(line.split(': C')[0])
            inRound = False
        if inARound:
            actualEndRoundLines.append(line.split(': C')[0])
            inARound = False
    elif 'Changing state from Playing to GameOver' in line: # 'Changing state from GameOver to Results'
        if inARound:
            inARound = False
            actualEndRoundLines.append(line.split(': [')[0])
    # overall show data
    elif '[CompletedEpisodeDto]' in line: # marker for a good show; only append show stats here
        partySizes.append(partySize)
        gameModes.append(gameMode)
        episodeMarkers.append(i)
        reg.append(to_add_reg)
        conne.append(to_add_conne)
        lookUser = True
        if inARound:
            actualEndRoundLines.append('left')
            inARound = False

# append last # achieving obj
numLines.append(prevNumLine.split('=')[-1])

# if no episodes found, end
if len(episodeMarkers) == 0:
    print('no episodes found') # change
        
# **********************************************************        
# get time user spent in each round ************************        
# (time round starts until they either finish or are eliminated) (just qualify I think)
# **********************************************************
roundTimes = []
for a, b in zip(startRoundLines, userEndRoundLines):
    roundTimes.append(subtractHours(a, b))

# get total round time
actualRoundTimes = []
for a, b in zip(startRoundLines, actualEndRoundLines):
    try:
        actualRoundTimes.append(subtractHours(a, b))
    except ValueError:
        actualRoundTimes.append('uncertain')

# gets start times for each show
startTimes = getStartTimes(reg, conne)


# **********************************************************
# for each show/episode ************************************
# **********************************************************
roundIdx = 0
showsSaved = 0
showsSkipped = 0

rnds = getExtraRoundInfoLines(possLines)
    
for showIdx, (j, user) in enumerate(zip(episodeMarkers, usernames)):
    this_show = total_shows
    total_shows += 1
    
    # get lines for this show
    final_lines = getShowLines(lines, j)
    
    # split data
    showData, rounds = roundSplit(final_lines)
    
    # set show data
    show_dict = {}
    show_dict['Show ID'] = this_show # id
    show_dict['Start Time'] = startTimes[showIdx]
    show_dict['Season'] = getSeason() 
    show_dict['Time Taken'] = getTimeTaken(startTimes[showIdx], final_lines[0].split(': ==')[0]) # approximate time taken
    show_dict['Game Mode'] = gameModes[showIdx]
    show_dict['Final'] = rounds[-1][0].split('|')[-1].strip() in list_of_finals # final
    show_dict['Rounds'] = len(rounds) # num rounds
    show_dict['Username'] = user
    show_dict['Party Size'] = partySizes[showIdx]
    show_dict['addID'] = final_lines[0].split(': ==')[0] # end time
    
    # add other show data
    for line in showData:
        show_dict[line.split(':')[0]] = line.split(':')[1].strip()
    
    # ********************************************
    # get data for each round in show ************
    # ********************************************
    rounds_list = []
    # for each round in the show/episode
    for round_ in rounds: # for list in 2D list
        round_dict = {'Show ID': this_show, 
                      'Round Num': round_[0].split(' ')[1].strip(), 
                      'Map': round_[0].split('|')[1].strip()}
        round_dict['Time Spent'] = roundTimes[roundIdx]
        round_dict['Round Length'] = actualRoundTimes[roundIdx]
        
        # add rest of data from list
        for line in round_[1:]:
            round_dict[line.split(':')[0]] = line.split(':')[1].strip()
        
        # add extra information
        rnd = rnds[roundIdx]
        splts = rnd.split()
        round_dict['Participants'] = splts[4].split('=')[-1] # num people
        round_dict['Qualification Percent'] = splts[7].split('=')[-1].replace(',', '') # qual %
        round_dict['Actual Num Qual'] = numLines[roundIdx]
        
        roundIdx += 1
        rounds_list.append(round_dict)
    
    # ********************************************
    # save ***************************************
    # ******************************************** 
    # save show_dict to one table
    # save each dict in rounds_list to another table
    if not saveData(show_dict, rounds_list):
        showsSkipped += 1
        total_shows -= 1
    else:
        showsSaved += 1

print('csvs saved successfully with {} new shows while skipping {} shows that were already saved'.format(showsSaved, showsSkipped))

with open('totalshows.txt', 'w') as f:
    f.write(str(total_shows))

# save processed lines
with open(os.path.join('data', 'archive', 'session{}.txt'.format(session_num)), 'w') as f:
    f.write("\n".join(lines))
session_num += 1

with open(os.path.join('data', 'session.txt'), 'w') as f:
    f.write(str(session_num))

first time...creating csvs
csvs saved successfully with 12 new shows while skipping 0 shows that were already saved


In [8]:
for line in lines:
    if 'Party Size' in line or 'Begin matchmaking solo' in line:
        print(line)
    if '[Round' in line:
        print(line)
    if '[CompletedEpisodeDto]' in line:
        print(line)

02:36:34.602: [StateMatchmaking] Begin matchmaking solo
02:48:36.962: == [CompletedEpisodeDto] ==
[Round 0 | round_biggestfan]
[Round 1 | round_chompchomp]
[Round 2 | round_wall_guys]
[Round 3 | round_dodge_fall]
[Round 4 | round_jump_showdown]
02:48:50.755: [StateMatchmaking] Begin matchmaking solo
02:56:41.370: == [CompletedEpisodeDto] ==
[Round 0 | round_door_dash]
[Round 1 | round_gauntlet_01]
[Round 2 | round_wall_guys]
[Round 3 | round_hoops_blockade_solo]
02:56:55.978: [StateMatchmaking] Begin matchmaking solo
03:09:05.132: == [CompletedEpisodeDto] ==
[Round 0 | round_biggestfan]
[Round 1 | round_lava]
[Round 2 | round_hoops]
[Round 3 | round_tail_tag]
03:10:16.170: [StateMatchmaking] Begin matchmaking solo
03:15:41.990: == [CompletedEpisodeDto] ==
[Round 0 | round_see_saw]
[Round 1 | round_ballhogs]
03:15:58.860: [StateMatchmaking] Begin matchmaking solo
03:28:57.886: == [CompletedEpisodeDto] ==
[Round 0 | round_gauntlet_02]
[Round 1 | round_biggestfan]
[Round 2 | round_ballhog

In [9]:
shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
shows_df

,Show ID,Start Time,Season,Time Taken,Game Mode,Final,Rounds,Username,Party Size,addID,Kudos,Fame,Crowns
0,644,2020-12-04 20:36:51.133,2,11:45.829000,main_show,True,5,Fast Swimming Panther,1,02:48:36.962,510,230,0
1,645,2020-12-04 20:48:53.545000,2,07:47.825000,main_show,False,4,Fast Swimming Panther,1,02:56:41.370,335,153,0
2,646,2020-12-04 20:57:02.340000,2,12:02.792000,main_show,False,4,Fast Swimming Panther,1,03:09:05.132,370,171,0
3,647,2020-12-04 21:10:27.122000,2,05:14.868000,main_show,False,2,Fast Swimming Panther,1,03:15:41.990,50,25,0
4,648,2020-12-04 21:16:01.673000,2,12:56.213000,main_show,True,5,Fast Swimming Panther,1,03:28:57.886,405,178,0
5,649,2020-12-04 21:29:18.553000,2,14:45.050000,main_show,True,5,Fast Swimming Panther,1,03:44:03.603,1175,399,1
6,650,2020-12-04 21:45:01.153000,2,12:29.995000,main_show,True,5,Fast Swimming Panther,1,03:57:31.148,1140,382,1
7,651,2020-12-04 21:58:18.832000,2,08:22.958000,main_show,True,4,Fast Swimming Panther,1,04:06:41.790,300,136,0
8,652,2020-12-04 22:07:12.025000,2,03:13.503000,main_show,False,2,Fast Swimming Panther,1,04:10:25.528,85,43,0
9,653,2020-12-04 22:11:01.279000,2,01:58.801000,main_show,False,1,Fast Swimming Panther,1,04:13:00.080,30,15,0


In [10]:
rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
rounds_df

,Show ID,Round Num,Map,Time Spent,Round Length,Qualified,Position,Kudos,Fame,Bonus Tier,Bonus Kudos,Bonus Fame,BadgeId,Participants,Qualification Percent,Actual Num Qual,Team Score,03
0,644,0,round_biggestfan,0:01:31.253000,0:03:06.255000,True,12,30,15,1.0,70,35,silver,60,73,43.0,NaN,NaN
1,644,1,round_chompchomp,0:00:30.770000,0:00:37.153000,True,6,20,10,1.0,70,35,silver,41,73,29.0,NaN,NaN
2,644,2,round_wall_guys,0:00:54.503000,0:02:18.987000,True,2,60,20,1.0,70,35,silver,29,60,17.0,NaN,NaN
3,644,3,round_dodge_fall,0:00:36.071000,0:00:49.071000,True,3,50,20,1.0,70,35,silver,17,61,10.0,NaN,NaN
4,644,4,round_jump_showdown,0:00:49.803000,uncertain,False,5,70,25,NaN,0,0,NaN,10,0,0.0,NaN,NaN
5,645,0,round_door_dash,0:00:51.255000,0:00:53.520000,True,27,30,15,2.0,35,18,bronze,60,73,43.0,NaN,NaN
6,645,1,round_gauntlet_01,0:00:43.336000,0:00:55.886000,True,4,20,10,1.0,70,35,silver,40,73,29.0,NaN,NaN
7,645,2,round_wall_guys,0:01:05.470000,0:01:25.171000,True,2,60,20,1.0,70,35,silver,29,60,17.0,NaN,NaN
8,645,3,round_hoops_blockade_solo,0:01:23.909000,0:01:21.889000,False,14,50,20,NaN,0,0,NaN,17,60,10.0,NaN,NaN
9,646,0,round_biggestfan,0:01:10.237000,0:03:16.940000,True,5,30,15,1.0,70,35,silver,58,73,42.0,NaN,NaN


In [11]:
# https://fallguysultimateknockout.gamepedia.com/Rounds

roundCodeDict = {
    'round_tip_toe': 'Tip Toe',
    'round_block_party': 'Block Party',
    'round_tunnel': 'Roll Out', 
    'round_egg_grab_02': 'Egg Siege', 
    'round_dodge_fall': 'Fruit Chute',
    'round_jump_showdown': 'Jump Showdown',
    'round_door_dash': 'Door Dash',
    'round_jump_club': 'Jump Club',
    'round_biggestfan': 'Big Fans',
    'round_wall_guys': 'Wall Guys',
    'round_match_fall': 'Perfect Match',
    'round_lava': 'Slime Climb',
    'round_gauntlet_03': 'The Whirlygig',
    'round_egg_grab': 'Egg Scramble',
    'round_see_saw': 'See Saw',
    'round_ballhogs': 'Hoarders', 
    'round_gauntlet_01': 'Hit Parade',
    'round_rocknroll': "Rock 'N' Roll",
    'round_fall_mountain_hub_complete': 'Fall Mountain',
    'round_gauntlet_04': 'Knight Fever',
    'round_jinxed': 'Jinxed',
    'round_chompchomp': 'Gate Crash',
    'round_conveyor_arena': 'Team Tail Tag', 
    'round_floor_fall': 'Hex-A-Gone',
    'round_tail_tag': 'Tail Tag',
    'round_hoops': 'Hoopsie Daisy',
    'round_hoops_blockade_solo': 'Hoopsie Legends', 
    'round_gauntlet_02': 'Dizzy Heights',
    'round_fall_ball_60_players': 'Fall Ball',
    # Royal Fumble
}

showTypeDict = {
    'main_show': 'Main Show',
    'event_only_survival_3010_to_0511': 'Slime Survivors',
    'event_only_season_2_variation_1011_to_1611': 'Medieval Mix Up',
    'event_only_hard_mode_2111_to_2711': 'Hard Mode',
    
}

In [12]:
# if running sessionX.txt, don't rerun preprocessing... (bool if session in argument path)
def splitMe(x):
    return int(x.split('.txt')[0].split('session')[1])

ss = sorted(os.listdir(os.path.join('personal', 'data', 'archive')), key=splitMe)
for s in ss:
    fs = os.path.join('data', 'archive', s)
    #print(fs)